# About dataset
* 99Acres dataset contains the data of different cities data is divided on the basis of cities
* I have only taken 2 cities to train the model so that it can be done in less computation

In [146]:
#Importing data of city gurgaon
import pandas as pd
gurgaon = pd.read_csv('/content/gurgaon.csv')
gurgaon.head(5)

<ipython-input-146-ec3951cc739b>:2: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  gurgaon = pd.read_csv('/content/gurgaon_10k.csv')


,PROP_ID,PHOTO_URL,MEDIUM_PHOTO_URL,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,LOCALITY,TRANSACT_TYPE,OWNTYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,FACING,AGE,FLOOR_NUM,TOTAL_FLOOR,FEATURES,REGISTER_DATE,PROP_NAME,MIN_PRICE,MAX_PRICE,PRICE_SQFT,LISTING,BUILDING_ID,CARPET_SQFT,SUPERBUILTUP_SQFT,BROKERAGE,MAP_DETAILS,FSL_Data,MIN_AREA_SQFT,MAX_AREA_SQFT,FORMATTED,AMENITIES,TOP_USPS,PD_URL,EXPIRY_DATE,GROUP_NAME,AREA,PRICE,PROP_HEADING,PROP_DETAILS_URL,CLASS_HEADING,CLASS_LABEL,SECONDARY_TAGS,PROPERTY_IMAGES,THUMBNAIL_IMAGES,TOTAL_LANDMARK_COUNT,FORMATTED_LANDMARK_DETAILS,CONTACT_NAME,CONTACT_COMPANY_NAME,DEALER_PHOTO_URL,SOCIETY_NAME,BUILDING_NAME,CITY_ID,LOCALITY_WO_CITY,profile,xid,metadata,location,BUILTUP_SQFT,SUPER_SQFT,COMMON_FURNISHING_ATTRIBUTES,QUALITY_SCORE,FURNISHING_ATTRIBUTES
0,O71585872,https://mediacdn.99acres.com/media1/22409/13/4...,https://mediacdn.99acres.com/media1/22409/13/4...,S,4 bhk flat for resale in alpha corp gurgaon one,Residential Apartment,Gurgaon,Sector 84 Gurgaon,1.0,1,4.0,4,4.0,8766.0,4,1,2,14,26.0,"33,23,12,46,25,47,39,1,3,6,9,40,41,30,20,42,21,32","29th Sep, 2023",Alpha Corp GurgaonOne 84,26300000,26300000,8766.0,R,10532,3000.0,3434.0,263000.0,"{'LATITUDE': '28.4065341', 'LONGITUDE': '76.96...",{'LOCALITY_ID': 0},319.028902,319.028902,"{'AVAIL': {'AVAILABILITY': 2, 'SUB_AVAILABILIT...","20,21,32,23,46,47,39,1,6,9,40,41,30,42,33,12,2...","['North Facing', 'Recently Renovated', 'Gated ...",/4-bhk-bedroom-apartment-flat-for-sale-in-alph...,2023-11-28 00:00:00,SPID_GROUP_SEARCH_NORMAL,3434 sq.ft.,2.63 Cr,4 BHK Flat in Sector 84 Gurgaon,4-bhk-bedroom-apartment-flat-for-sale-in-alpha...,Dealer: propertyline,Dealer,"['READY TO MOVE', 'RESALE']",['https://mediacdn.99acres.com/media1/22409/13...,['https://mediacdn.99acres.com/media1/22409/13...,10.0,"[{'category': 'Shopping', 'text': '4 Shoppings...",gaurav khosla,propertyline,https://static.99acres.com/images/dealer_pnava...,Alpha Corp GurgaonOne 84,Alpha Corp GurgaonOne 84,8,Sector 84,"{'CONTACT_NAME': 'gaurav khosla', 'CONTACT_COM...","{'PROJECTID': 5096, 'PROJ_NAME': 'Alpha Corp G...","{'PROP_PHOTO_COUNT': '4', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",NaN,NaN,NaN,NaN,NaN
1,T71585466,https://mediacdn.99acres.com/media1/22425/6/44...,https://mediacdn.99acres.com/media1/22425/6/44...,S,"Explore dlf ultima, an upscale ready-To-Move h...",Residential Apartment,Gurgaon,Sector 81 Gurgaon,1.0,1,4.0,4,3.0,21176.0,4,1,1,7,29.0,"44,33,23,12,46,25,47,26,17,29,19,1,3,5,6,9,30,...","29th Sep, 2023",DLF The Ultima,36000000,36000000,21176.0,R,12704,1700.0,2870.0,1.0,"{'LATITUDE': '28.3889069', 'LONGITUDE': '76.95...",{'LOCALITY_ID': 0},266.631610,266.631610,"{'AVAIL': {'AVAILABILITY': 2, 'SUB_AVAILABILIT...","17,5,20,21,10,44,23,46,47,29,19,1,6,9,30,41,43...","['North Facing', ""st. xavier's within 500m"", '...",/4-bhk-bedroom-apartment-flat-for-sale-in-dlf-...,2023-11-28 00:00:00,SPID_GROUP_SEARCH_NORMAL,2870 sq.ft.,3.6 Cr,4 BHK Flat in Sector 81 Gurgaon,4-bhk-bedroom-apartment-flat-for-sale-in-dlf-t...,Dealer: MVD AND ASSOCIATES,Dealer,"['READY TO MOVE', 'RESALE', 'RERA']",['https://mediacdn.99acres.com/media1/22425/6/...,['https://mediacdn.99acres.com/media1/22425/6/...,9.0,"[{'category': 'MetroStation', 'text': '1 Metro...",Kamal,MVD AND ASSOCIATES,https://static.99acres.com/images/dealer_pnava...,DLF The Ultima,DLF The Ultima,8,Sector 81,"{'CONTACT_NAME': 'Kamal', 'CONTACT_COMPANY_NAM...","{'PROJECTID': 7182, 'PROJ_NAME': 'DLF The Ulti...","{'PROP_PHOTO_COUNT': '4', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",NaN,NaN,NaN,NaN,NaN
2,D71549524,https://mediacdn.99acres.com/media1/22386/13/4...,https://mediacdn.99acres.com/media1/22386/13/4...,R,A 3 bhk flat for rent in sector 66 gurgaon clo...,Residential Apartment,Gurgaon,Sector 66 Gurgaon,NaN,0,3.0,3,2.0,43.0,1,4,1,7,17.0,"44,23,45,24,46,25,47,28,29,30,31,32,33,12,17,3...","27th Sep, 2023",Emaar MGF The Palm Drive,8500

In [147]:
# Dropping useless column
gurgaon.drop(columns = ['PHOTO_URL', 'MEDIUM_PHOTO_URL', 'EXPIRY_DATE', 'PD_URL', 'PROPERTY_IMAGES', 'THUMBNAIL_IMAGES', 'CONTACT_NAME', 'CONTACT_COMPANY_NAME', 'DEALER_PHOTO_URL', 'profile', 'xid', 'metadata', 'location'], inplace = True)

In [148]:
# Dropping useless column
gurgaon.drop(columns = ['COMMON_FURNISHING_ATTRIBUTES','QUALITY_SCORE','FURNISHING_ATTRIBUTES','BUILDING_ID', 'PROP_ID','PROP_DETAILS_URL','FSL_Data','TRANSACT_TYPE'], inplace = True)

In [149]:
# Cleaning Locality column
gurgaon['LOCALITY'] = gurgaon['LOCALITY'].str.split('Gurgaon').str.get(0).str.strip()

In [150]:
# function used to find the BHK of house
def get_bhk(text):
  pattern = r'(\d+)(?=\s*bhk)'
  x = list(set(re.findall(pattern, text)))
  if len(x) == 0:
    return None
  else:
    return x[0]

In [151]:
import re
text = gurgaon['DESCRIPTION'].apply(lambda x: get_bhk(x))
gurgaon['BHK'] = text

In [152]:
gurgaon['BHK'] = gurgaon['BHK'].astype(float)

In [153]:
# Extracting longitude and latitude From Map_details
import ast
Lat = []
Long = []
for d in gurgaon['MAP_DETAILS']:
    lat = ast.literal_eval(d)['LATITUDE']
    long = ast.literal_eval(d)['LONGITUDE']
    Lat.append(lat)
    Long.append(long)

In [154]:
gurgaon['LATITUDE'] = pd.Series(Lat).astype(float)
gurgaon['LONGITUDE'] = pd.Series(Long).astype(float)

In [155]:
# Cleaning Area column
gurgaon['AREA'] = gurgaon['AREA'].str.split(' ', expand=True).get(0)

In [156]:
# Dropping useless column
gurgaon.drop(columns = ['CITY_ID','PREFERENCE','DESCRIPTION', 'FEATURES','REGISTER_DATE','LISTING','BROKERAGE','FORMATTED','TOP_USPS','GROUP_NAME'], inplace = True)

In [157]:
indx = gurgaon[gurgaon['PRICE'] == 'Price on Request']

In [158]:
gurgaon.drop(indx.index, inplace=True)

In [159]:
# function to refine Price data
def price(item):
  if 'L' in item:
    return float(item.replace('L', '').replace('.','').strip())*100000
  elif 'Cr' in item:
    return float(item.replace('Cr', '').strip())*10000000
  elif ',' in item:
    return float(item.replace(',','').strip())
  else:
    return float(item)

In [160]:
# function to refine Price data
def p(item):
  if '-' in item:
    return item.split('-')[1]
  else:
    return item

In [161]:
gurgaon['PRICE'] = gurgaon['PRICE'].apply(p).apply(price)

In [162]:
# Dropping useless column
gurgaon.drop(columns = ['MIN_PRICE','MAX_PRICE','MIN_AREA_SQFT','MAX_AREA_SQFT','PROP_HEADING','CLASS_HEADING','CLASS_LABEL','SECONDARY_TAGS','TOTAL_LANDMARK_COUNT','FORMATTED_LANDMARK_DETAILS','SOCIETY_NAME','BUILDING_NAME','LOCALITY_WO_CITY','BUILTUP_SQFT','SUPER_SQFT'], inplace = True)

In [163]:
# Dropping useless column
gurgaon.drop(columns = ['MAP_DETAILS'], inplace = True)

In [164]:
# Dropping useless column
gurgaon.drop(columns=['PRICE_PER_UNIT_AREA'],inplace = True)

In [165]:
# calculating total sccore of amenities
def calculate_luxury_score(amenities):
    # Define amenity scores (adjust as needed)
    amenity_scores = {
        'Waste Disposal' : 1,
        'Rain Water Harvesting' : 3,
        'Bank Attached Property' : 2,
        'Power Back-up' : 4,
        'Feng Shui / Vaastu Compliant' : 5,
        'Private Garden / Terrace' : 8,
        'Centrally Air Conditioned' : 9,
        'Security / Fire Alarm' : 7,
        'Intercom Facility' : 6,
        'Water Storage' : 2,
        'Piped-gas' : 3,
        'Water purifier' : 4,
        'Near Bank' : 1,
        'Swimming Pool' : 9,
        'Club house / Community Center' : 8,
        'Park' : 5,
        'Security Personnel' : 7,
        'Fitness Centre / GYM' : 8,
        'Visitor Parking' : 4,
        'Lift(s)' : 9,
        'Maintenance Staff' : 6,
        'Shopping Centre' : 7,
        'WheelChair Accessibility' : 5,
        'DG Availability' : 8,
        'CCTV Surveillance' : 6,
        'Grade A Building' : 9,
        'Grocery Shop' : 3,
        'ATM' : 1,
        'Cafeteria / Food Court' : 7,
        'Bar / Lounge' : 9,
        'Conference room' : 8,
        'Service / Goods Lift' : 6,
        'Access to High Speed Internet' : 9
        # Add other amenities and their scores here
    }

    # Calculate total luxury score for the property
    total_score = sum(amenity_scores.get(amenity, 1) for amenity in amenities)

    return total_score

def categorize_property(total_score):
    if total_score >= 15:
        return "Luxury"
    elif 8 <= total_score <= 14:
        return "Semi-luxury"
    else:
        return "Budget"


luxury_score = gurgaon['PROPERTY_TYPE'].apply(calculate_luxury_score)
gurgaon['LUXURY'] = luxury_score.apply(categorize_property)
print(f"Total Luxury Score: {luxury_score}")
print(f"Category: {gurgaon['LUXURY']}")

Total Luxury Score: 0        21
1        21
2        21
3        21
4        21
5        21
6        21
7        21
8        16
9        25
10       16
11       21
12       21
13       21
14       21
15       16
16       21
17       21
18       21
19       21
20       21
21       21
22       25
23       21
24       21
25       21
26       21
27       21
28       21
29       25
30       21
31       21
32       23
33       23
34       21
35       21
36       21
37       23
38       25
39       25
40       25
41       25
42       25
43       25
44       21
45       21
46       21
47       21
48       21
49       21
50       21
51       21
52       21
53       21
54       21
55       21
56       21
57       21
58       21
59       21
60       21
61       21
62       21
63       21
64       21
65       21
66       21
67       21
68       21
69       21
70       21
71       21
72       21
73       21
74       21
75       21
76       21
77       21
78       21
79       21
80       21
81      

In [166]:
# Dropping useless column
gurgaon.drop(columns = ['AMENITIES'], inplace = True)

In [167]:
# Dropping useless column
gurgaon.drop(columns=['PROP_NAME'],inplace = True)

In [168]:
# Dropping useless column
gurgaon.drop(columns=['BATHROOM_NUM'], inplace=True)

In [169]:
# Dropping duplicates
gurgaon.drop_duplicates(inplace = True)

In [170]:
gurgaon[(gurgaon['CARPET_SQFT'] == None) & (gurgaon['SUPERBUILTUP_SQFT'] == None)]

,PROPERTY_TYPE,CITY,LOCALITY,OWNTYPE,BEDROOM_NUM,BALCONY_NUM,FURNISH,FACING,AGE,FLOOR_NUM,TOTAL_FLOOR,PRICE_SQFT,CARPET_SQFT,SUPERBUILTUP_SQFT,AREA,PRICE,BHK,LATITUDE,LONGITUDE,LUXURY


In [171]:
gurgaon.to_csv('gurgaon_cleaned.csv',index = True)

In [172]:
gurgaon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10662 entries, 0 to 10703
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROPERTY_TYPE      10662 non-null  object 
 1   CITY               10662 non-null  object 
 2   LOCALITY           10662 non-null  object 
 3   OWNTYPE            10662 non-null  int64  
 4   BEDROOM_NUM        10470 non-null  float64
 5   BALCONY_NUM        10463 non-null  float64
 6   FURNISH            10662 non-null  int64  
 7   FACING             10662 non-null  int64  
 8   AGE                10662 non-null  int64  
 9   FLOOR_NUM          9768 non-null   object 
 10  TOTAL_FLOOR        10655 non-null  float64
 11  PRICE_SQFT         10662 non-null  float64
 12  CARPET_SQFT        6517 non-null   float64
 13  SUPERBUILTUP_SQFT  6685 non-null   float64
 14  AREA               10662 non-null  object 
 15  PRICE              10662 non-null  float64
 16  BHK                7187 non

In [201]:
am = pd.read_excel('/content/amenities.xlsx')
# am.reset_index(drop=True, inplace=True)
am.set_index('id')

,category,type,label
id,,,
25,Other Features,FEATURES_RESIDENTIAL,Waste Disposal
26,Other Features,FEATURES_RESIDENTIAL,Rain Water Harvesting
28,Other Features,FEATURES_COMMERCIAL,Bank Attached Property
2,Property Feature,FEATURES_COMMERCIAL,Power Back-up
5,Property Feature,FEATURES_RESIDENTIAL,Feng Shui / Vaastu Compliant
8,Property Feature,FEATURES_RESIDENTIAL,Private Garden / Terrace
10,Property Feature,FEATURES_COMMERCIAL,Centrally Air Conditioned
17,Property Feature,FEATURES_RESIDENTIAL,Security / Fire Alarm
20,Property Feature,FEATURES_RESIDENTIAL,Intercom Facility


In [174]:
import pandas as pd

In [175]:
pd.set_option('display.max_columns', None)

In [176]:
pd.set_option('display.max_rows', None)

In [177]:
# importing hyderbad dataset
hyderbad = pd.read_csv('/content/hyderabad.csv')
hyderbad.head(1)

,SPID,PROP_ID,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,TRANSACT_TYPE,OWNTYPE,BEDROOM_NUM,PRICE_PER_UNIT_AREA,FURNISH,FACING,AGE,TOTAL_FLOOR,FEATURES,REGISTER_DATE,POSTING_DATE,UPDATE_DATE,CLASS,RES_COM,PROP_NAME,PROPERTY_NUMBER,MIN_PRICE,MAX_PRICE,PRICE_SQFT,PROJ_ID,BUILDING_ID,VERIFIED,MAP_DETAILS,MIN_AREA_SQFT,MAX_AREA_SQFT,AMENITIES,ALT_TAG,PRODUCT_TYPE,TOP_USPS,EXPIRY_DATE,PROPERTY_TYPE__U,AREA,SECONDARY_AREA,PRICE,PROP_HEADING,VALUE_LABEL,CLASS_HEADING,CLASS_LABEL,REGISTER_DATE__U,REGISTERED_DAYS,PRIMARY_TAGS,SECONDARY_TAGS,TOTAL_LANDMARK_COUNT,FORMATTED_LANDMARK_DETAILS,SOCIETY_NAME,BUILDING_NAME,location,BALCONY_NUM,FLOOR_NUM
0,70135726,Z70135726,S,"Hmda, rera approved residential apartments at ...",Residential Apartment,Secunderabad,1.0,1,2.0,5700.0,2,4,1,5.0,"23,24,5,17,6,20,21","08th Jul, 2023",1.688834e+12,1689100520000,A,R,Evolve,1,6925500,6925500,5700.0,0,0,N,"{'LATITUDE': '17.575957', 'LONGITUDE': '78.471...",112.877145,112.877145,"5,17,20,21,23,24,6","₹69.25 Lac, 2 bhk Residential Apartment in Gun...",PREMIUM_LISTING,NaN,2023-09-06 00:00:00,1,1215 sq.ft.,112.88 sq.m.,69.25 L,2 BHK Flat in Gundlapochampalli,Freehold,Dealer: Rainbow Developers,Dealer,2023-07-08 22:01:14,1 month ago,['FEATURED'],"['READY TO MOVE', 'RESALE']",9.0,[{'text': '1 Library'}],Evolve,Evolve,"{'CITY': '268', 'CITY_NAME': 'Secunderabad', '...",1.0,2


In [178]:
#dropping all empty rows in the column AMENITES
hyderbad = hyderbad[hyderbad['AMENITIES'].notna()]

In [179]:
import re
def bhk(text):
  pattern = r'(\d+)(?=\s*BHK)'
  x = re.findall(pattern, text)
  if len(x) == 0:
    return None
  else:
    return x

In [180]:
hyderbad['BHK'] = hyderbad['PROP_HEADING'].apply(bhk).str.get(0)

In [181]:
import ast
text = []
for i in hyderbad['location']:
  text = (ast.literal_eval(i)['LOCALITY_NAME'])
hyderbad['LOCALITY'] = text

In [182]:
import ast
Lat = []
Long = []
for d in hyderbad['MAP_DETAILS']:
    lat = ast.literal_eval(d)['LATITUDE']
    long = ast.literal_eval(d)['LONGITUDE']
    Lat.append(lat)
    Long.append(long)

In [183]:
hyderbad['LATITUDE'] = pd.Series(Lat).astype(float)
hyderbad['LONGITUDE'] = pd.Series(Long).astype(float)

In [184]:
hyderbad['AMENITIES'] = hyderbad['AMENITIES'].str.split(',').tolist()

In [185]:
def calculate_luxury_score(amenities):
    # Define amenity scores (adjust as needed)
    amenity_scores = {
        '1' : 1,
        '2' : 3,
        '3' : 2,
        '4' : 4,
        '5' : 5,
        '6' : 8,
        '7' : 9,
        '8' : 7,
        '9' : 6,
        '10' : 2,
        '11' : 3,
        '12' : 4,
        '13' : 1,
        '14' : 9,
        '15' : 8,
        '16' : 5,
        '17' : 7,
        '18' : 8,
        '19' : 4,
        '20' : 9,
        '21' : 6,
        '22' : 7,
        '23' : 5,
        '24' : 8,
        '25' : 6,
        '26' : 9,
        '27' : 3,
        '28' : 1,
        '29' : 7,
        '30' : 9,
        '31' : 8,
        '32' : 6,
        '33' : 9
        # Add other amenities and their scores here
    }

    # Calculate total luxury score for the property
    if amenities == None:
      return 1

    total_score = sum(amenity_scores.get(amenity, 1) for amenity in amenities)

    # for i in range(len(amenities)):
    #   total_score += amenity_scores[amenities[i]]

    return total_score

def categorize_property(total_score):
    if total_score >= 15:
        return "Luxury"
    elif 8 <= total_score <= 14:
        return "Semi-luxury"
    else:
        return "Budget"


luxury_score = hyderbad['AMENITIES'].apply(calculate_luxury_score)
hyderbad['LUXURY'] = luxury_score.apply(categorize_property)
# print(f"Total Luxury Score: {luxury_score}")
# print(f"Category: {['LUXURY']}")

In [186]:
hyderbad.rename(columns = {'SECONDARY_AREA' : 'SUPERBUILTUP_SQFT'}, inplace=True)

In [187]:
hyderbad['AREA'] = hyderbad['AREA'].str.split(' ', expand=True).get(0).astype(float)

In [188]:
hyderbad['SUPERBUILTUP_SQFT'] = hyderbad['SUPERBUILTUP_SQFT'].str.split(' ', expand=True).get(0).astype(float)*10.7639104

In [189]:
hyderbad['CARPET_SQFT'] = hyderbad['AREA'] - abs(hyderbad['AREA'] - hyderbad['SUPERBUILTUP_SQFT'])

In [190]:
hyderbad.drop(columns=['AMENITIES','PRICE_PER_UNIT_AREA','location','FEATURES','MAP_DETAILS','DESCRIPTION','PREFERENCE','SPID','PROP_ID','TRANSACT_TYPE','REGISTER_DATE','POSTING_DATE','UPDATE_DATE','CLASS','RES_COM','PROP_NAME','PROPERTY_NUMBER','MIN_PRICE','MAX_PRICE','PROJ_ID','BUILDING_ID','VERIFIED','MIN_AREA_SQFT','MAX_AREA_SQFT','PRODUCT_TYPE','TOP_USPS','EXPIRY_DATE','PROPERTY_TYPE__U','ALT_TAG','PROP_HEADING','VALUE_LABEL','CLASS_HEADING','CLASS_LABEL','REGISTER_DATE__U','REGISTERED_DAYS','PRIMARY_TAGS','SECONDARY_TAGS','TOTAL_LANDMARK_COUNT','FORMATTED_LANDMARK_DETAILS','SOCIETY_NAME','BUILDING_NAME'], inplace=True)

In [191]:
hyderbad = hyderbad[hyderbad['PRICE'].str.contains('Bed') == False]

In [192]:
hyderbad = hyderbad[hyderbad['PRICE'] != 'Price on Request']

In [193]:
def bed(item):
  return float(item['PRICE'].str.split('/').get(0))*item['BEDROOM_NUM']

In [194]:
hyderbad['PRICE'] = hyderbad['PRICE'].apply(price)

In [195]:
hyderbad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8058 entries, 0 to 9486
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROPERTY_TYPE      8058 non-null   object 
 1   CITY               8058 non-null   object 
 2   OWNTYPE            8058 non-null   int64  
 3   BEDROOM_NUM        5812 non-null   float64
 4   FURNISH            8058 non-null   int64  
 5   FACING             8058 non-null   int64  
 6   AGE                8058 non-null   int64  
 7   TOTAL_FLOOR        7655 non-null   float64
 8   PRICE_SQFT         8058 non-null   float64
 9   AREA               8058 non-null   float64
 10  SUPERBUILTUP_SQFT  8058 non-null   float64
 11  PRICE              8058 non-null   float64
 12  BALCONY_NUM        5812 non-null   float64
 13  FLOOR_NUM          4395 non-null   object 
 14  BHK                4372 non-null   object 
 15  LOCALITY           8058 non-null   object 
 16  LATITUDE           6809 non-n

In [196]:
hyderbad.to_csv('hyderbad_cleaned.csv',index = True)

In [197]:
data = pd.concat([hyderbad, gurgaon])

In [198]:
data.head(1)

,PROPERTY_TYPE,CITY,OWNTYPE,BEDROOM_NUM,FURNISH,FACING,AGE,TOTAL_FLOOR,PRICE_SQFT,AREA,SUPERBUILTUP_SQFT,PRICE,BALCONY_NUM,FLOOR_NUM,BHK,LOCALITY,LATITUDE,LONGITUDE,LUXURY,CARPET_SQFT
0,Residential Apartment,Secunderabad,1,2.0,2,4,1,5.0,5700.0,1215.0,1215.030206,692500000.0,1.0,2,2,Manikonda,17.575957,78.471167,Luxury,1214.969794


In [200]:
data.to_csv('99Acres_gurgaon_hyderabad.csv')